In [1]:
import tensorflow as tf

In [2]:
x = tf.Variable(1.0, dtype = tf.float32)

In [3]:
#在tf.function中用input_signature限定输入张量的签名类型：shape和dtype

@tf.function(input_signature = [tf.TensorSpec(shape = [], dtype = tf.float32)])
def add_print(a):
    x.assign_add(a)
    tf.print(x)
    return(x)

In [4]:
add_print(tf.constant(3.0)) 

4


<tf.Tensor: shape=(), dtype=float32, numpy=4.0>

In [5]:
add_print(tf.constant(3))
#不符合张量签名的参数将会报错

ValueError: Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor(3, shape=(), dtype=int32))
  input_signature: (
    TensorSpec(shape=(), dtype=tf.float32, name=None)).

# tf.Module子类化包装 

In [16]:
class DemoModule(tf.Module):
    def __init__(self,init_value = tf.constant(0.0), name = None):
        super(DemoModule, self).__init__(name = name)
        with self.name_scope: ##相当于 with tf.name_scope('demo_module')  保证变量名字unique
            self.x = tf.Variable(init_value, dtype = tf.float32, trainable = True)
            
    @tf.function(input_signature = [tf.TensorSpec(shape = [], dtype = tf.float32)])
    def addprint(self,a):
        with self.name_scope:
            self.x.assign_add(a)
            tf.print(self.x)
            return(self.x)

In [21]:
demo = DemoModule(init_value = tf.constant(1.0))
result = demo.addprint(tf.constant(5.0))

6


In [23]:
print(demo.variables)
print(demo.trainable_variables)

(<tf.Variable 'demo_module/Variable:0' shape=() dtype=float32, numpy=6.0>,)
(<tf.Variable 'demo_module/Variable:0' shape=() dtype=float32, numpy=6.0>,)


In [24]:
demo.submodules

()

In [25]:
tf.saved_model.save(demo,"./data/demo/1",signatures = {"serving_default":demo.addprint})


INFO:tensorflow:Assets written to: ./data/demo/1\assets


In [29]:
demo2 = tf.saved_model.load("./data/demo/1")
demo2.addprint(tf.constant(5.0))


11


<tf.Tensor: shape=(), dtype=float32, numpy=11.0>

In [38]:
import datetime

#创建日志
stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = './data/demomodule/%s' % stamp
sriter = tf.summary.create_file_writer(logdir)

#开启autograph跟踪
tf.summary.trace_on(graph = True, profiler = True)

#执行autograph
demo = DemoModule(init_value = tf.constant(0.0))
result = demo.addprint(tf.constant(5.0))

with writer.as_default():
    tf.summary.trace_export(
        name="demomodule",
        step=0,
        profiler_outdir=logdir)

5


NameError: name 'writer' is not defined

In [36]:
%reload_ext tensorboard

In [37]:
from tensorboard import notebook
notebook.list() 

Known TensorBoard instances:
  - port 6006: logdir /home/kesci/input/data3483/data/autograph/ (started 2 days, 5:03:06 ago; pid 3260)


In [39]:
notebook.start("--logdir /home/kesci/input/data3483/data/demomodule/")


# 除了子类化封装，我们也可以通过给tf.Module添加属性的方式进行封装

In [64]:
mymodule = tf.Module()
mymodule.x = tf.Variable(1.0)

@tf.function(input_signature = [tf.TensorSpec(shape = [], dtype = tf.float32)])
def addprint(a):
    mymodule.x.assign_add(a)
    tf.print(mymodule.x)
    return(mymodule.x)

mymodule.addprint = addprint



In [65]:
mymodule.addprint(tf.constant(2.0)).numpy()

3


3.0

In [66]:
print(mymodule.variables)


(<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=3.0>,)


In [67]:
#使用tf.saved_model 保存模型
tf.saved_model.save(mymodule,"/home/kesci/input/data3483/data/mymodule",
    signatures = {"serving_default":mymodule.addprint})

#加载模型
mymodule2 = tf.saved_model.load("/home/kesci/input/data3483/data/mymodule")
mymodule2.addprint(tf.constant(5.0))

INFO:tensorflow:Assets written to: /home/kesci/input/data3483/data/mymodule\assets
8


<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

# tf.Module和tf.keras.Model, tf.keras.layers.Layer

In [68]:
from tensorflow.keras import models,layers,losses,metrics

In [69]:
print(issubclass(tf.keras.Model,tf.Module))
print(issubclass(tf.keras.layers.Layer,tf.Module))
print(issubclass(tf.keras.Model,tf.keras.layers.Layer))

True
True
True


In [96]:
##为什么有55个Param？ 因为有10个factors加上一个bias总共11个factors, 同样18=3*(5+1)
tf.keras.backend.clear_session()
model = models.Sequential()
model.add(layers.Dense(5, input_shape = (10,)))
model.add(layers.Dense(3))
model.add(layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 5)                 55        
                                                                 
 dense_1 (Dense)             (None, 3)                 18        
                                                                 
 dense_2 (Dense)             (None, 1)                 4         
                                                                 
Total params: 77
Trainable params: 77
Non-trainable params: 0
_________________________________________________________________


In [97]:
model.variables

[<tf.Variable 'dense/kernel:0' shape=(10, 5) dtype=float32, numpy=
 array([[ 0.3934005 ,  0.01091367,  0.37369996,  0.5337382 ,  0.01126981],
        [-0.33413035, -0.19943869, -0.34478572, -0.5195776 ,  0.27706403],
        [-0.0953331 , -0.10554922,  0.2859968 , -0.25736195,  0.32303828],
        [ 0.41709715, -0.22836143, -0.4954242 , -0.31989118,  0.46800107],
        [ 0.21515548, -0.12004387,  0.5423551 ,  0.3367613 , -0.45232522],
        [ 0.29215127, -0.11387599,  0.2654996 ,  0.36095905,  0.1043331 ],
        [ 0.6164548 ,  0.00668734,  0.4739445 , -0.13507178,  0.55378884],
        [-0.15039197,  0.3711266 ,  0.41191965,  0.4987722 ,  0.41416007],
        [ 0.56227237, -0.51478225,  0.62021154,  0.62141937,  0.2140907 ],
        [-0.21711919,  0.411555  , -0.0021174 , -0.08928889,  0.05563623]],
       dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(5,) dtype=float32, numpy=array([0., 0., 0., 0., 0.], dtype=float32)>,
 <tf.Variable 'dense_1/kernel:0' shape=(5, 3) dtype=

In [99]:
#冻结第0层的变量使其不可训练
model.layers[0].trainable = False
model.trainable_variables

[<tf.Variable 'dense_1/kernel:0' shape=(5, 3) dtype=float32, numpy=
 array([[ 0.84073204,  0.22384042,  0.33428305],
        [ 0.3861702 ,  0.49348122,  0.85640126],
        [-0.58224964,  0.2365461 ,  0.5541727 ],
        [-0.7203611 ,  0.37333828, -0.4322285 ],
        [-0.6467968 ,  0.03510165,  0.84442836]], dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>,
 <tf.Variable 'dense_2/kernel:0' shape=(3, 1) dtype=float32, numpy=
 array([[-0.39697254],
        [ 0.84431994],
        [ 0.96303356]], dtype=float32)>,
 <tf.Variable 'dense_2/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]

In [100]:
model.submodules


(<keras.engine.input_layer.InputLayer at 0x1b619677c40>,
 <keras.layers.core.dense.Dense at 0x1b61967f9a0>)

In [101]:
model.layers

In [102]:
print(model.name)
print(model.name_scope())

sequential
sequential/
